# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv data and create a dataframe.
cities_df = pd.read_csv('../WeatherPy/data/city_weather.csv')
# Drop the NA that were inserted for cities without weather info
cities_df.dropna(inplace=True)

In [3]:
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hermanus,-34.418696,19.234464,55.00,72.0,32.0,5.99,ZA,1.600038e+09
1,Cape Town,-33.925839,18.423218,55.99,81.0,75.0,11.41,ZA,1.600037e+09
2,Touros,-5.200000,-35.466667,75.20,83.0,15.0,9.17,BR,1.600038e+09
3,New Norfolk,-42.780750,147.059387,48.00,64.0,29.0,5.01,AU,1.600038e+09
4,Puerto Ayora,-0.750000,-90.316667,68.00,89.0,85.0,5.99,EC,1.600038e+09
...,...,...,...,...,...,...,...,...,...
995,Corrales,5.829683,-72.843321,84.00,15.0,1.0,5.82,PE,1.600038e+09
996,Hirara,24.803788,125.302222,78.80,94.0,75.0,2.24,JP,1.600038e+09
997,Damghan,36.168300,54.348000,62.64,51.0,0.0,2.82,IR,1.600038e+09
998,Burley,42.535833,-113.791944,89.60,11.0,90.0,5.82,US,1.600038e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
coordinates = cities_df[['Lat','Lng']]

# store the figure in a variable so we can continue to add layers to the same map
fig = gmaps.figure()


# Store 'Poverty Rate' in a variable called 'poverty_rate'
humidity_data = cities_df['Humidity'].astype(float)

# Create the heatmap layer using locations and poverty_rate
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_data, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

weather_criteria_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70) & 
              (cities_df['Wind Speed'] < 10) & 
             (cities_df['Cloudiness'] == 0)]

print(f"The list of cities matching the criteria contains {len(weather_criteria_df)} cities.")
weather_criteria_df.reset_index(drop=True, inplace=True)
weather_criteria_df

The list of cities matching the criteria contains 20 cities.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Iranshahr,27.202446,60.684758,79.59,16.0,0.0,6.22,IR,1.600038e+09
1,Yerkoy,39.638056,34.467222,70.66,49.0,0.0,3.80,TR,1.600038e+09
2,Paranaiba,-19.666667,-51.183333,79.59,24.0,0.0,2.57,BR,1.600038e+09
3,Deoband,29.700000,77.683333,78.06,74.0,0.0,3.74,IN,1.600038e+09
4,Mantua,22.288889,-84.284444,71.01,82.0,0.0,5.82,CU,1.600038e+09
5,Kikwit,-5.038611,18.818056,78.62,55.0,0.0,1.63,CD,1.600038e+09
6,Richards Bay,-28.775417,32.054005,72.09,63.0,0.0,8.50,ZA,1.600038e+09
7,Nkhotakota,-12.916667,34.300000,72.34,58.0,0.0,3.44,MW,1.600038e+09
8,Yol,32.166667,76.200000,74.14,59.0,0.0,3.74,IN,1.600038e+09
9,Frutal,-20.033333,-48.916667,79.39,20.0,0.0,7.09,BR,1.600038e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_criteria_df

#### Setup the framework for the API call

* We are going to narrow the search for the first 10 cities.

In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in weather_criteria_df.iloc[0:10,:].iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    print(f"Searching within {row['City']}, {row['Country']}...")

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)


    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Searching within Iranshahr, IR...


/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Searching within Yerkoy, TR...
Searching within Paranaiba, BR...
Searching within Deoband, IN...
Searching within Mantua, CU...
Searching within Kikwit, CD...
Searching within Richards Bay, ZA...
Searching within Nkhotakota, MW...
Searching within Yol, IN...
Searching within Frutal, BR...


In [8]:
# Drop the rows that contains NA leaving only the 10 hotels in the list with the desired columns
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Hotel Name']].dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Iranshahr,IR,27.202446,60.684758,خوابگاه یاسر ریگیyaser.rigi
1,Yerkoy,TR,39.638056,34.467222,Yerköy Öğretmenevi
2,Paranaiba,BR,-19.666667,-51.183333,Villa Rica Park Hotel GMR
3,Deoband,IN,29.700000,77.683333,Anupam Hotel
4,Mantua,CU,22.288889,-84.284444,Hotel Mangos De Roques
5,Kikwit,CD,-5.038611,18.818056,Guest L'Amour
6,Richards Bay,ZA,-28.775417,32.054005,BON Hotel Waterfront Richards Bay
7,Nkhotakota,MW,-12.916667,34.300000,Sitima Inn
8,Yol,IN,32.166667,76.200000,Hotel Gopal
9,Frutal,BR,-20.033333,-48.916667,Premium Hotel


In [9]:
# Export the hotels dataframe as csv
hotel_df.to_csv('data/hotels_list.csv',index=False)

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))